# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
#import gmaps 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
csv_path = "city_data.csv" 
city_data_df = pd.read_csv(csv_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
cordinates = city_data_df[['Latitude','Longitude']]
cordinates

,Latitude,Longitude
0,56.40,61.88
1,55.10,-105.28
2,38.42,-121.42
3,42.00,8.92
4,67.45,153.68
...,...,...
592,-6.43,39.55
593,37.94,22.96
594,54.01,33.96
595,37.57,126.98


In [4]:
figure_layout = {
    'width' : '400px',
    'height' : '300px',
    'border' : '1px solid black',
    'padding' :'1px',
    'margin' : '0 auto 0 auto'
}

fig = gmaps.figure(layout = figure_layout)

In [6]:
markers = gmaps.marker_layer(cordinates)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
city_data_df = city_data_df.loc[city_data_df['Temperature °F']>70]
city_data_df = city_data_df.loc[city_data_df['Temperature °F']<80]
city_data_df = city_data_df.loc[city_data_df['Wind Speed']<10]
city_data_df = city_data_df.loc[city_data_df['Cloudiness']==0]

In [93]:
hotel_df = city_data_df.dropna()
hotel_df

,Unnamed: 0,City,Temperature °F,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
19,19,Gerāsh,77.00,38,0,3.20,27.67,54.14
111,111,San Quintín,73.87,64,0,8.01,30.48,-115.95
166,166,Mitha Tiwana,79.97,29,0,4.97,32.25,72.11
208,208,Iaçu,78.33,53,0,8.59,-12.77,-40.21
304,304,Nuevo Ideal,77.65,23,0,6.93,24.89,-105.07
305,305,Caravelas,73.42,79,0,8.25,-17.71,-39.25
402,402,Ujhāni,77.68,52,0,0.34,28.02,79.02
502,502,Santa Catarina de Tepehuanes,79.30,23,0,7.99,25.35,-105.73
529,529,Abu Samrah,71.96,42,0,8.77,35.30,37.18
531,531,Linhares,73.89,78,0,8.25,-19.39,-40.07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [90]:
import requests 

In [94]:
hotel_cordinates = zip(hotel_df['Latitude'],hotel_df['Longitude'])
#hotel_cordinates = 
target_radius = 5000
target_type = 'lodging'

hotel_name = []

for hotel_cordinates in hotel_cordinates:
    
    try:
        params = {
            'location' : (f'{hotel_cordinates[0]}, {hotel_cordinates[1]}'),
            'radius' : target_radius,
            'type' : target_type,
            'key' : g_key
        }

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        response = requests.get(base_url, params=params)
        hotel_data = response.json()
        hotel_name.append(hotel_data['results'][0]['name'])
    
    except IndexError:
        print(f'{hotel_cordinates[0]}, {hotel_cordinates[1]} Does not exist')

35.3, 37.18 Does not exist
30.15, 62.6 Does not exist


In [97]:
exist_hotel_df = hotel_df.drop([529,573])
exist_hotel_df['Hotel Name'] = hotel_name

In [98]:
exist_hotel_df

,Unnamed: 0,City,Temperature °F,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
19,19,Gerāsh,77.00,38,0,3.20,27.67,54.14,Chelpasin
111,111,San Quintín,73.87,64,0,8.01,30.48,-115.95,Old Mill Hotel
166,166,Mitha Tiwana,79.97,29,0,4.97,32.25,72.11,Haveli Malik Sultan Ali Tiwana
208,208,Iaçu,78.33,53,0,8.59,-12.77,-40.21,Pousada e Restaurante Portal da Chapada
304,304,Nuevo Ideal,77.65,23,0,6.93,24.89,-105.07,Hotel Las Águilas
305,305,Caravelas,73.42,79,0,8.25,-17.71,-39.25,Pousada dos Navegantes
402,402,Ujhāni,77.68,52,0,0.34,28.02,79.02,Bhagwati Palace
502,502,Santa Catarina de Tepehuanes,79.30,23,0,7.99,25.35,-105.73,hotel plaza
531,531,Linhares,73.89,78,0,8.25,-19.39,-40.07,Hotel Conceição


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
